In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Pitching Airfoil 

In [2]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from time import time
from jax.debug import print as jprint

# Jax 64-bit
import jax

jax.config.update("jax_enable_x64", True)

In [3]:
from test_integrators import test_all_integrators, plot_results

# Dynamical Systems

In [4]:
from ICARUS.dynamical_systems.first_order_system import NonLinearSystem
from ICARUS.dynamical_systems.second_order_system import SecondOrderSystem
from ICARUS.dynamical_systems.integrate import (
    BackwardEulerIntegrator,
    ForwardEulerIntegrator,
    RK4Integrator,
    RK45Integrator,
    CrankNicolsonIntegrator,
    GaussLegendreIntegrator,
    NewmarkIntegrator,
)

In [ ]:
# Define a simple m-c-k system
L = 8
I = 1
R = 2.0
K = 10
d = 0.5
rho = 1.225
U = 0.5
c = 2


theta_0 = 1 * np.pi / 180
crank_y = R * np.cos(theta_0)
crank_x = R * np.sin(theta_0)
y_equilibrium = np.sqrt(L**2 - R**2 * np.cos(theta_0) ** 2) + R * np.sin(theta_0)

DL = (
    np.sqrt((y_equilibrium - R * np.sin(theta_0)) ** 2 + R**2 * np.cos(theta_0) ** 2)
    - L
)
print(DL)

In [ ]:
y_equilibrium

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)

airf = DB.get_airfoil("S3010")
polar = DB.foils_db.get_polars("S3010", "Xfoil")
cl, cd, cm = polar.get_aero_coefficients(polar.reynolds_nums[0], 0.0)
a_0 = polar.get_reynolds_zero_lift_angle(polar.reynolds_nums[0])
print(a_0)
a_0 = 0

In [ ]:
xs = {}
ts = {}

In [ ]:
def y_input(t: float) -> jnp.ndarray:
    return y_equilibrium + 0.2 * jnp.sin(2 * jnp.pi / 10 * t)


def f(t: float, x: jnp.ndarray) -> jnp.ndarray:
    y = y_input(t)
    th = x[0]
    v = x[1]  # x' = v

    sin = jnp.sin(th)
    cos = jnp.cos(th)

    a = -(
        L * R * K * y * cos / jnp.sqrt(R**2 - y * (2 * R * sin - y)) - R * K * y * cos
    )

    # # Theodorsen method for the lift coefficient
    l_ec = 1 / 2
    ksi = (l_ec - 3 / 4) * c

    th_th = th
    v_th = v
    a_th = a

    W_eff2 = (-U * jnp.cos(0) + jnp.sin(th_th) * ksi * v_th) ** 2 + (
        U * jnp.sin(0) + jnp.cos(-th_th) * ksi * v_th
    ) ** 2
    W_eff = jnp.sqrt(W_eff2)
    a_eff = (
        -jnp.arctan(
            (U * jnp.sin(0) + jnp.cos(th_th) * ksi * v_th)
            / (-U * jnp.cos(0) + jnp.sin(th_th) * ksi * v_th)
        )
        - th_th
    )

    A1 = 0.165
    A2 = 0.335
    b1 = 0.0455
    b2 = 0.3

    y1 = x[2]
    y2 = x[3]

    C_k_epi_a = a_eff * (1 - A1 - A2) + y1 + y2
    cl = 2 * jnp.pi * (C_k_epi_a - a_0) + jnp.pi * c / (2 * W_eff2) * (
        -W_eff * v_th - (1 / 2 - l_ec) * a_th * c
    )
    cm = (4 * l_ec - 1) * jnp.pi / 2 * C_k_epi_a - jnp.pi / (2 * W_eff2) * (
        -(3 / 4 - l_ec) * W_eff * v_th * c
        - 1 / 4 * (9 / 8 + 4 * l_ec**2 - 4 * l_ec) * a_th * c**2
    )

    a_y1 = -(b1 * 2 * W_eff / c * y1 - b1 * A1 * 2 * W_eff / c * a_eff)
    a_y2 = -(b2 * 2 * W_eff / c * y2 - b2 * A2 * 2 * W_eff / c * a_eff)
    cl, cd, cm = polar.get_aero_coefficients(
        polar.reynolds_nums[0], a_eff * 180 / jnp.pi
    )

    # PRINT
    # jprint("t= {time}", time = t)
    # jprint("W_eff = {x}", x=W_eff)
    # jprint('Th = {x}', x=th*180/jnp.pi)
    # jprint('A_eff = {x}', x=a_eff*180/jnp.pi)
    # jprint('a = {x}', x=a)
    # jprint('v = {x}', x=v)
    # jprint("CL = {x}",x=cl)
    # jprint('cm = {x}', x=cm)
    # jprint('')

    # # Q is the aerodynamic moment
    # Q = 0.5 * rho * W_eff**2 * c * (cm)
    Q = 0.5 * rho * W_eff**2 * c * (cl * d + cm)
    return jnp.array(
        [
            v,  # x' = v
            a + 0 * Q / I,  # v' = a = f(t, x)/ m
            a_y1,  # y1
            a_y2,  # y2
        ]
    )


# Create the system
system = NonLinearSystem(f)

# Test the integrators
dt = 1e-6 * 5
x0 = jnp.array([theta_0, 0.0])
x0 = jnp.array([theta_0, 0.0, 0.0, 0.0])
x_data, t_data = test_all_integrators(system, x0, 0.0, 40, dt, compare_with_scipy=True)

In [ ]:
xs["pol_cl_cm"] = x_data["Gauss-Legendre"]
ts["pol_cl_cm"] = t_data["Gauss-Legendre"]

In [ ]:
xs3 = {
    k: v
    for k, v in xs.items()
    if k
    in [
        "theo_cl_cm",
        "pol_cl_cm",
    ]
}
ts3 = {
    k: v
    for k, v in ts.items()
    if k
    in [
        "theo_cl_cm",
        "pol_cl_cm",
    ]
}

plot_results(xs3, ts3)

In [ ]:
xs.keys()

In [ ]:
x = jnp.array([0.1, 0.0])
y = y_input(0)

th = x[0]
v = x[1]  # x' = v

a = (
    jnp.sin(th)
    * R
    * K
    * y
    * (1 - L / jnp.sqrt(R**2 - 2 * R * y * jnp.cos(th) + y**2))
    / I
)

print(jnp.sin(th) * R * K * y)

# Frequency Analysis

In [ ]:
from scipy.fft import fft, fftfreq

In [ ]:
x = x_data[list(x_data.keys())[0]][:, 0].tolist()
t = t_data[list(x_data.keys())[0]][:].tolist()


# Perform FFT
fft_result = fft(x)
freqs = fftfreq(len(x), t[1] - t[0])

# Find dominating frequencies
n = len(x)
mask = freqs > 0
fft_result_magnitude = 2.0 / n * np.abs(fft_result)
dominant_freqs = freqs[mask][np.argsort(fft_result_magnitude[mask])][::-1]

# Plot original time series data
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# plt.plot(t, x)
# plt.title("Original Time Series Data")
# plt.xlabel("Time")
# plt.ylabel("Amplitude")

# Plot FFT result
plt.subplot(1, 2, 2)
plt.plot(freqs[mask], fft_result_magnitude[mask])
plt.title("FFT Analysis")
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.xlim(0, max(dominant_freqs) * 2)  # Adjust x-axis limit for better visualization

# Mark dominating frequencies
# for freq in dominant_freqs:
#     plt.axvline(x=freq, color='r', linestyle='--')

plt.show()

# Print dominating frequencies
# print("Dominant Frequencies:")
# for freq in dominant_freqs:
#     print("{:.2f} Hz".format(freq))

In [ ]:
# FFt of the displacement
x = x_data[:, 1].tolist()
t = t_data.tolist()
T = t[1] - t[0]

N = len(x)
yf = fft(x)
xf = fftfreq(N, T)[: N // 2]
# Remove the DC component
yf[0] = 0

# Find the peak
idx = np.argmax(np.abs(yf[0 : N // 2]))
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(xf, 2.0 / N * np.abs(yf[0 : N // 2]))
# Add a vertical line at the peak
ax.axvline(xf[idx], color="r", linestyle="--")
ax.set_title("FFT of the displacement")
ax.set_xlabel("Frequency (Hz)")
# Add text to the plot
ax.text(
    xf[idx],
    2.0 / N * np.abs(yf[idx]),
    f"Peak at {xf[idx]*(2*np.pi)} Hz",
    verticalalignment="bottom",
)